In [45]:
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [46]:
import requests                     # hacer request a la pagina
import pandas as pd                 # dataframes
import datetime  
from datetime import datetime                   # formato de fechas
from datetime import timedelta

Carga inicial de datos
Por medio de este link
https://service.iris.edu/fdsnws/event/1/query?starttime=2022-11-14T00:00:00&endtime=2022-11-22T00:00:00&minmag=3&maxmag=10&includeallmagnitudes=true&orderby=time&format=geocsv&maxdepth=351&maxlat=45.557&minlon=122.932&maxlon=153.986&minlat=20.9325&nodata=404

In [47]:
inicio = "2010-01-01T00:00:00"
carga= "2022-11-23T00:00:00"
link = "https://service.iris.edu/fdsnws/event/1/query?starttime="+ inicio + "&endtime="+ carga + "&minmag=3&maxmag=10&includeallmagnitudes=true&orderby=time&format=geocsv&maxdepth=351&maxlat=45.557&minlon=122.932&maxlon=153.986&minlat=20.9325&nodata=404"

In [48]:
df_jp = pd.read_csv(link, sep='|', skiprows=4)
df_jp = df_jp[df_jp.EventLocationName.str.contains('JAPAN')]

/var/folders/4h/zd1h7yk93j37j92519m4z_y80000gn/T/ipykernel_61486/79469537.py:1: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  df_jp = pd.read_csv(link, sep='|', skiprows=4)


In [49]:
df_jp.head()

,EventID,Time,Latitude,Longitude,Depth,Author,Catalog,Contributor,ContributorID,MagType,Magnitude,MagAuthor,EventLocationName
0,11624461,2022-11-22T16:06:39Z,31.4649,138.4152,2.242,us,NEIC PDE,us,us7000irj8,mb,4.6,us,"SOUTHEAST OF HONSHU, JAPAN"
3,11623688,2022-11-20T20:19:08Z,37.0886,141.0882,61.378,us,NEIC PDE,us,us7000ir7f,mb,4.5,us,"NEAR EAST COAST OF HONSHU, JAPAN"
4,11626113,2022-11-20T12:16:20Z,32.3621,142.4036,10.000,us,NEIC PDE,us,us7000is4w,mb,4.5,us,"SOUTHEAST OF HONSHU, JAPAN"
5,11623519,2022-11-20T09:17:24Z,42.9154,144.6995,52.949,us,NEIC PDE,us,us7000ir4k,Mwr,4.6,us,"HOKKAIDO, JAPAN REGION"
7,11623016,2022-11-19T01:39:34Z,44.7684,140.9380,264.857,us,NEIC PDE,us,us7000iqw6,mb,4.1,us,EASTERN SEA OF JAPAN


date	     time	   mag	depth	lon	lat	idLocation	country	geoReference
2022-08-17	0:47:09

In [50]:
def sep_date(date):
    part = date.split("T")
    return part[0]
def sep_time(date):
    part = date.split("T")
    return part[1].strip("Z")
"""def pais(data):
    if data == None:
        return None
    return data.raw['address']['country']
def city(data):
    if data == None:
        return None
    return data.raw['address']['city']
def traduct(x):
    if x ==None:
        return None
    return traductor.translate(x)"""
"""def locat(string):
    if string.find(",") == -1:
        return "Sea"
    if string.find("NEAR S.")!= -1:
        return "Near South Coast of Honshu"
    string = string.split(",")
    return string[0].title()"""

def reg(string):
    string = string.upper()
    if string.find("HONSHU")!= -1:
        return "Honshu"
    if string.find("HOKKAIDO")!= -1:
        return "Hokkaido"
    if string.find("SHIKOKU")!= -1:
        return "Shikoku"
    if string.find("KYUSHU")!= -1:
        return "Kyushu"
    if string.find("RYUKYU")!= -1:
        return "Ryukyu"
    if string.find("SEA")!= -1:
        return "Sea"
    return None
def format_str(df, column):
    '''Formatear strings colocando iniciales en mayuscula 
    y eliminando espacios en blanco a izquiera y derecha'''

    return df[column].astype(str).str.strip().str.title()

In [51]:
df_state = pd.read_csv(r'./datasets/estados.csv')
df_state.drop('Unnamed: 0', axis=1, inplace=True)
df_state = df_state.reset_index()
df_state.rename({'index':'idLocation','State':'state', 'iso2':'country'}, axis='columns', inplace=True)
def normIRIS(DataFrame):
    if str(type(DataFrame)) != "<class 'pandas.core.frame.DataFrame'>":
        print("Falla en la carga, inserte en el primer argumento un DataFrame de Pandas")
        return None
    DataFrame = DataFrame[["Time","Latitude","Longitude","Depth","Magnitude","EventLocationName"]]
    #Acomodamos las fechas
    DataFrame["date"]=DataFrame["Time"].apply(sep_date)
    DataFrame["time"]=DataFrame["Time"].apply(sep_time)
    #Filtramos la profundidad y mag
    DataFrame = DataFrame[DataFrame["Depth"]<351]
    DataFrame = DataFrame[DataFrame["Magnitude"]>=3]
    DataFrame = DataFrame[["date","time","Latitude","Longitude","Depth","Magnitude","EventLocationName"]]
    #renombramos columnas
    DataFrame.rename(columns={"Latitude":"latitude","Longitude":"longitude","Depth":"depth","EventLocationName":"geoRef","Magnitude":"magnitude"},inplace=True)
    states_jp = df_state[df_state.country == 'JP']
    DataFrame.geoRef = format_str(DataFrame,'geoRef')
    DataFrame["region"]= DataFrame["geoRef"].apply(reg)
    DataFrame = pd.merge(DataFrame, states_jp, left_on= ['region'], right_on= ['state'], how = 'left')
    DataFrame = DataFrame[~((DataFrame.state.isna())|(DataFrame.state == 'Sea'))]
    DataFrame = DataFrame[['date','time','magnitude','depth','longitude','latitude','idLocation',"country",'geoRef']]
    
    return DataFrame

In [52]:
df_state

,idLocation,state,country
0,0,Alabama,US
1,1,Alaska,US
2,2,Arizona,US
3,3,Arkansas,US
4,4,California,US
...,...,...,...
83,83,Honshu,JP
84,84,Shikoku,JP
85,85,Kyushu,JP
86,86,Okinawa,JP


date	   time   mag	depth	lon	lat	idLocation	country	geoReference

In [53]:
df_prueba = normIRIS(df_jp)

/var/folders/4h/zd1h7yk93j37j92519m4z_y80000gn/T/ipykernel_61486/818243390.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  DataFrame["date"]=DataFrame["Time"].apply(sep_date)
/var/folders/4h/zd1h7yk93j37j92519m4z_y80000gn/T/ipykernel_61486/818243390.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  DataFrame["time"]=DataFrame["Time"].apply(sep_time)


In [54]:
df_jp.shape

(63650, 13)

In [55]:
df_prueba.shape

(53426, 9)

In [56]:
df_prueba

,date,time,magnitude,depth,longitude,latitude,idLocation,country,geoRef
0,2022-11-22,16:06:39,4.6,2.242,138.4152,31.4649,83.0,JP,"Southeast Of Honshu, Japan"
1,2022-11-20,20:19:08,4.5,61.378,141.0882,37.0886,83.0,JP,"Near East Coast Of Honshu, Japan"
2,2022-11-20,12:16:20,4.5,10.000,142.4036,32.3621,83.0,JP,"Southeast Of Honshu, Japan"
3,2022-11-20,09:17:24,4.6,52.949,144.6995,42.9154,82.0,JP,"Hokkaido, Japan Region"
5,2022-11-17,17:35:19,4.6,56.678,141.8036,38.1835,83.0,JP,"Near East Coast Of Honshu, Japan"
...,...,...,...,...,...,...,...,...,...
63418,2010-01-01,22:44:49,3.4,120.500,139.8187,35.0328,83.0,JP,"Near S. Coast Of Honshu, Japan"
63421,2010-01-01,15:41:07,3.9,31.000,144.6900,38.3400,83.0,JP,"Off East Coast Of Honshu, Japan"
63422,2010-01-01,09:21:57,3.9,43.400,142.6779,40.7639,83.0,JP,"Near East Coast Of Honshu, Japan"
63423,2010-01-01,02:42:14,3.4,267.100,141.5028,44.7342,82.0,JP,"Hokkaido, Japan Region"


In [57]:
#df_prueba.to_csv("JP_IRIS_2000-2022")

# Carga incremental

In [58]:
savedate = carga
now = datetime.now() - timedelta(hours=15) #le agregue un delay de 10 horas porque no cargaba, hay que ir haciendolo mas chico
#acomodamos para que respete el formato del link
year = str(now.year)
if len(str(now.month))<2:
    month = "0" + str(now.month)
else:
    month = str(now.month)
if len(str(now.day))<2:
    day = "0" + str(now.day)
else:
    day = str(now.day)
if len(str(now.hour))<2:
    hour = "0"  + str(now.hour)
else:
    hour = str(now.hour)
if len(str(now.minute))<2:
    minute = "0"  + str(now.minute)
else:
    minute = str(now.minute)
if len(str(now.second))<2:
    second = "0"  + str(now.second)
else:
    second = str(now.second)

loadnow = year + "-" + month + "-" + day + "T" + hour + ":" + minute + ":" + second
link_increm = "https://service.iris.edu/fdsnws/event/1/query?starttime=" + savedate + "&endtime="+ loadnow + "&minmag=3&maxmag=10&includeallmagnitudes=true&orderby=time&format=geocsv&maxdepth=351&maxlat=45.557&minlon=122.932&maxlon=153.986&minlat=20.9325&nodata=404"
savedate = loadnow

In [59]:
len(str(now.month))

2

In [60]:
link_increm

'https://service.iris.edu/fdsnws/event/1/query?starttime=2022-11-23T00:00:00&endtime=2022-11-27T21:44:59&minmag=3&maxmag=10&includeallmagnitudes=true&orderby=time&format=geocsv&maxdepth=351&maxlat=45.557&minlon=122.932&maxlon=153.986&minlat=20.9325&nodata=404'

In [61]:
df_jp2 = pd.read_csv(link_increm, sep='|', skiprows=4)
df_jp2 = df_jp2[df_jp2.EventLocationName.str.contains('JAPAN')]
df_jp2 = normIRIS(df_jp2)
df_jp2.head() #pa cargar

/var/folders/4h/zd1h7yk93j37j92519m4z_y80000gn/T/ipykernel_61486/818243390.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  DataFrame["date"]=DataFrame["Time"].apply(sep_date)
/var/folders/4h/zd1h7yk93j37j92519m4z_y80000gn/T/ipykernel_61486/818243390.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  DataFrame["time"]=DataFrame["Time"].apply(sep_time)


,date,time,magnitude,depth,longitude,latitude,idLocation,country,geoRef
0,2022-11-27,21:40:46,4.6,49.711,142.8466,41.1883,82.0,JP,"Hokkaido, Japan Region"
1,2022-11-26,20:00:19,4.9,15.983,142.4756,32.3345,83.0,JP,"Southeast Of Honshu, Japan"
2,2022-11-26,17:45:57,4.9,18.799,142.5084,32.4496,83.0,JP,"Southeast Of Honshu, Japan"
3,2022-11-26,15:17:36,4.5,305.443,138.8676,31.1923,83.0,JP,"Southeast Of Honshu, Japan"
4,2022-11-26,08:30:00,4.5,59.748,142.6249,42.0365,82.0,JP,"Hokkaido, Japan Region"
